## Liberias y word embedding

In [247]:
import pandas as pd 
import numpy as np
import nltk
from string import punctuation 
import re
from sklearn import svm
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from gensim.models import KeyedVectors
from numpy.linalg import norm # para normalizar datos
from nltk.tokenize import word_tokenize

In [248]:
representacionVectorial = KeyedVectors.load_word2vec_format('archivos/fasttext-sbwc.3.6.e20.vec', limit=100000)

## Tokenize 

In [249]:
def Tokenize(text):
        tokens = nltk.word_tokenize(text,  language='spanish')
        stems = []
        for item in tokens:
            stems.append(nltk.PorterStemmer().stem(item))
        return stems

## Limpiar Corpus

In [275]:
def LimpiarCorpus(X_frases):
    non_words = list(punctuation) 
    non_words.extend(['¿', '¡'])
    for i in range(len(X_frases)):
        if X_frases[i].lower() not in non_words:
            X_frases[i] = re.sub("[^a-zA-Z]", " ", X_frases[i].lower())
            X_frases[i] = ' '.join(X_frases[i].split())
    return X_frases

## Vectorizar Frase con word embeding 

In [276]:
def VectorizarWordEmbedding(frases):
    X_frases = []
    for frase in frases:
        tokens = Tokenize(frase)
        vec = np.zeros(300)
        palabrasEncontradas = 0
        for palabra in tokens:
            if palabra in representacionVectorial:
                palabrasEncontradas +=1
                vec += representacionVectorial[palabra]
        if palabrasEncontradas != 0:
            X_frases.append(vec / norm(vec))
        else:
            X_frases.append(vec)
    return np.asarray(X_frases)

## Entrenar y medir modelos ML

In [277]:
def EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo):
    # MODELO
    if(algoritmo=="SVM"):
        modelo = svm.SVC(kernel='linear') 
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Random Forest"):
        modelo=RandomForestClassifier(n_estimators=10)
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Naive Bayes"):
        modelo = GaussianNB()
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    # METRICAS
    #reporteMetricas= classification_report(Y_test, prediccion)
    return metrics.accuracy_score(Y_test, prediccion),modelo

## Clasificar palabra usando vector word Embedding como representacion

In [278]:
def ClasificarPolaridadWE(corpus,algoritmo):
    X_frases = np.asarray(corpus[['A']])
    X_frases = X_frases.ravel()
    Y_sentimiento = np.asarray(corpus[['B']])
    Y_sentimiento = Y_sentimiento.ravel()
    X_frases = LimpiarCorpus(X_frases)
    X_frases = VectorizarWordEmbedding(X_frases)
    # PARTIR CORPUS PARA ENTRENAR
    X_train, X_test, Y_train, Y_test = train_test_split(X_frases, Y_sentimiento, test_size=0.30)
    return EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo)

# MAIN

In [283]:
accuracyPromedio = 0
algoritmo = "SVM"
numeroEjecuciones = 100
for i in range(numeroEjecuciones):
    corpus = pd.read_csv('archivos/aspectos.csv')
    accuracy, modelo = ClasificarPolaridadWE(corpus,algoritmo)
    accuracyPromedio += accuracy
accuracyPromedio/numeroEjecuciones

0.8198141263940522

In [ ]:
0.7728156996587029

In [266]:
corpus = pd.read_csv('archivos/sentimientoOriginal.csv')
accuracy, modelo, reporte = ClasificarPolaridadWE(corpus,algoritmo)
for frase in corpus['A']:
    tokens = Tokenize(frase)
    print(tokens)

/home/manuel/anaconda3/envs/Sistemas_Recomendacion/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


['restaurant', 'acogedor', ',', 'con', 'comida', 'de', 'calidad', 'y', 'buen', 'servicio', '.']
['con', 'el', 'descuento', 'de', 'atrápalo', 'relación', 'calidad/precio', 'de', '10', ',', 'si', 'no', 'me', 'parecería', 'caro', '.']
['tanto', 'el', 'trato', 'com', 'restaurant', 'acogedor', ',', 'con', 'comida', 'de', 'calidad', 'y', 'buen', 'servicio', '.']
['tanto', 'el', 'trato', 'como', 'la', 'comida', ',', 'fueron', 'de', 'primera', 'categoría', ',', 'un', 'lugar', 'sin', 'duda', 'para', 'repetir', '.']
['comida', 'correcta', 'pero', 'un', 'poco', 'desangelado', 'el', 'local', 'ya', 'que', 'había', 'muy', 'poca', 'mesa', '.']
['menú', 'degustación', 'correcto', ',', 'sin', 'alard', '.']
['A', 'destacar', 'el', 'servicio', ',', 'muy', 'agrad', 'y', 'pendient', 'de', 'lo', 'detal', '.']
['La', 'comida', 'deliciosa', ',', 'tomamo', 'carpaccio', 'de', 'buey', 'como', 'entrant', 'y', 'albondiga', 'de', 'dorada', 'confit', 'de', 'pato', 'como', 'principal', ',', 'muy', 'bien', 'cocinado',

['No', 'no', 'gustó', 'la', 'comida', ',', 'la', 'servían', 'fría', ',', 'la', 'costilla', 'muy', 'seca', 'y', 'sin', 'salsa', '(', 'sólo', 'ketchup', 'y', 'mostaza', ')', '.']
['Lo', 'mejor', ',', 'el', 'arroz', 'al', 'horno', '.']
['En', 'el', 'menú', 'había', 'varia', 'opcion', 'de', 'postr', 'pero', 'en', 'el', 'restaurant', 'no', 'no', 'dieron', 'a', 'elegir', '.']
['ambient', 'y', 'atención', 'buena', '.']
['la', 'comida', 'bien', 'y', 'el', 'sitio', 'en', 'gener', 'merec', 'la', 'pena', '.']
['buena', 'cena', 'contundent', 'para', '2', 'pero', 'con', 'algo', 'de', 'demora', 'en', 'el', 'servicio', '.']
['El', 'servicio', 'de', 'la', 'camarera', 'bueno', 'y', 'lo', 'mejor', 'el', 'postr', '.']
['lo', 'camarero', 'muy', 'atento', ',', 'quizá', 'demasiado', 'pero', 'lo', 'sabor', 'eran', 'bastant', 'bueno']
['lugar', 'tranquilo', 'en', 'un', 'ambient', 'agrad', 'dond', 'se', 'pued', 'comer', 'bien', '.']
['Es', 'un', 'sitio', 'para', 'ir', 'con', 'frecuencia', 'pue', 'tien', 'varie